Partie 1 : Importation des bibliothèques et préparation des données
On commence par modulariser le code, en créant des fonctions pour des tâches spécifiques et en utilisant des paramètres pour rendre le code plus flexible.




In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import classification_report

# Définir les équipes Euro et Copa une seule fois
euro_teams = [
    'Albania', 'Germany', 'England', 'Austria', 'Belgium', 'Croatia', 'Denmark',
    'Scotland', 'Spain', 'France', 'Georgia', 'Hungary', 'Italy', 'Netherlands',
    'Poland', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Switzerland',
    'Czech Republic', 'Turkey', 'Ukraine'
]

copa_teams = [
    'Argentina', 'Peru', 'Chile', 'Brazil', 'United States', 'Colombia', 'Canada',
    'Mexico', 'Jamaica', 'Venezuela', 'Ecuador', 'Paraguay', 'Costa Rica',
    'Uruguay', 'Panama', 'Bolivia'
]

# Fonction pour charger et filtrer les données
def load_and_filter_data(file_path, euro_teams, copa_teams, start_date='2016-01-01'):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'])
    
    # Filtrer les matchs impliquant des équipes Euro ou Copa
    df_filtered = df[df['date'] >= start_date]
    df_filtered = df_filtered[(df_filtered['home_team'].isin(euro_teams + copa_teams)) |
                              (df_filtered['away_team'].isin(euro_teams + copa_teams))]
    
    # Créer une colonne de résultat : 0 (défaite), 1 (égalité), 2 (victoire)
    df_filtered['result'] = np.where(df_filtered['home_score'] > df_filtered['away_score'], 2,
                                     np.where(df_filtered['home_score'] == df_filtered['away_score'], 1, 0))
    
    return df_filtered

# Charger les données
file_path = '/content/drive/MyDrive/22102/results.csv'
df_filtered = load_and_filter_data(file_path, euro_teams, copa_teams)

# Afficher un aperçu des données filtrées
df_filtered.head()


Modularisation : La fonction load_and_filter_data permet de filtrer les données en fonction des équipes Euro/Copa et d'ajouter une colonne pour le résultat du match. Cela évite la répétition de code et rend le script plus clair.
Paramétrisation : Le chemin du fichier et la date de début sont passés en paramètres, ce qui rend le code plus flexible.


Partie 2 : Préparation des données pour le modèle LSTM
Ici, nous créons une fonction pour construire les séquences de données nécessaires à l'entraînement du modèle LSTM.

In [ ]:
# Fonction pour créer des séquences de matchs
def build_sequences(df, teams):
    sequences = {}
    for team in teams:
        # Séparer les matchs à domicile et à l'extérieur
        home_matches = df[df['home_team'] == team]['result'].values
        away_matches = df[df['away_team'] == team]['result'].values
        # Concaténer les séquences
        sequences[team] = np.concatenate((home_matches, away_matches))
    return sequences

# Créer des séquences pour les données d'entraînement et de test
train_data, test_data = train_test_split(df_filtered, test_size=0.2, shuffle=True, random_state=42)
train_sequences = build_sequences(train_data, euro_teams + copa_teams)
test_sequences = build_sequences(test_data, euro_teams + copa_teams)


La fonction build_sequences crée des séquences pour chaque équipe en combinant les résultats des matchs à domicile et à l'extérieur. Cela permet de passer ensuite ces séquences à un modèle LSTM.

Partie 3 : Préparation des données d'entrée pour le modèle
Ensuite, nous préparons les données d'entraînement et de test en utilisant les séquences construites précédemment.

In [ ]:
# Fonction pour préparer les données pour le modèle LSTM
def prepare_data(sequences, sequence_length):
    X, y = [], []
    for team, sequence in sequences.items():
        for i in range(len(sequence) - sequence_length):
            X.append(sequence[i:i+sequence_length])
            y.append(sequence[i+sequence_length])
    return np.array(X), np.array(y)

# Définir la longueur de la séquence
sequence_length = 10

# Préparer les données pour l'entraînement et le test
X_train, y_train = prepare_data(train_sequences, sequence_length)
X_test, y_test = prepare_data(test_sequences, sequence_length)

# Reshaper les données pour le LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


Modularisation : La fonction prepare_data permet de transformer les séquences en entrées et sorties exploitables par le modèle LSTM.
Reshape des données : Comme le modèle LSTM prend des entrées sous forme de séquences tridimensionnelles (nombre de séquences, longueur des séquences, 1 caractéristique), les données sont mises en forme en conséquence.


Partie 4 : Construction et entraînement du modèle LSTM
Nous construisons ensuite le modèle LSTM avec plusieurs couches de LSTM et de Dropout, comme dans ton code initial, mais ici, j'ajoute une amélioration des hyperparamètres.

In [ ]:
# Fonction pour construire le modèle LSTM
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.3))  # Légèrement augmenté pour réduire le sur-apprentissage
    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(units=64))
    model.add(Dropout(0.3))
    model.add(Dense(units=3, activation='softmax'))  # 3 classes de sortie (défaite, égalité, victoire)
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Construire et entraîner le modèle
model = build_lstm_model((X_train.shape[1], 1))
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Évaluation du modèle
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy du modèle sur les données de test :", accuracy)


Hyperparamètres : J'ai augmenté les unités LSTM de 50 à 64 et le taux de Dropout de 0.2 à 0.3 pour améliorer la robustesse du modèle. Ce sont des valeurs que tu peux ajuster selon les performances observées.


Partie 5 : Évaluation du modèle
En plus de l'accuracy, je te propose d'ajouter un rapport de classification pour mieux évaluer les résultats.

In [ ]:
# Prédictions sur les données de test
y_pred = np.argmax(model.predict(X_test), axis=1)

# Rapport de classification
print(classification_report(y_test, y_pred))


Classification Report : Cette fonction fournit des informations détaillées sur la précision, le rappel, et le F1-score, qui sont des métriques essentielles pour évaluer un modèle de classification multiclasses.
